In [2]:
import csv
import numpy as np
import pandas as pd
from dateutil import parser as dt_parse
from datetime import datetime
from sklearn.metrics import accuracy_score

from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor

data = pd.read_csv("/home/krullun/Documents/coding/JPMorgan/Problem_1/train.csv")
test_data = pd.read_csv("/home/krullun/Documents/coding/JPMorgan/Problem_1/test.csv")

In [9]:
data.

AttributeError: 'DataFrame' object has no attribute 'row'

In [8]:
city = data.From.unique().tolist()
flight_class = data.Class.unique().tolist() 

current_datetime = datetime.now()

mean = np.zeros((len(city), len(city)))
std_dev = np.zeros((len(city), len(city)))
minimum = np.zeros((len(city), len(city)))
maximum = np.zeros((len(city), len(city)))
for source in city:
	for destination in city:
		fares = data.Fare[(data['From'] == source) & (data['To'] == destination)]
		mean[city.index(source)][city.index(destination)] = fares.mean()
		std_dev[city.index(source)][city.index(destination)] = fares.std()
		minimum[city.index(source)][city.index(destination)] = fares.min()
		maximum[city.index(source)][city.index(destination)] = fares.max()
        
X_train = []
Y_train = []
models = []
for source in city:
    for destination in city:
        data.


for ind,row in data.iterrows():
    city_vec = np.zeros(len(city))
    city_vec[city.index(row['From'])] = 2
    city_vec[city.index(row['To'])] = 1

    age_vec = np.zeros(4)
    age = ((dt_parse.parse(row['Booking Date']) - dt_parse.parse(row['Date of Birth'])).days)/365
    if age < 12:
        age_vec[0] = 1
    elif age < 25:
        age_vec[1] = 1
    elif age < 50:
        age_vec[2] = 1
    else:
        age_vec[3] = 1

    duration = (dt_parse.parse(row['Flight Date']) - dt_parse.parse(row['Booking Date'])).days

    weekday_vec = np.zeros(7)
    weekday_ind = ((current_datetime - dt_parse.parse(row['Flight Date'])).days)%7
    weekday_vec[weekday_ind] = 1

    time = dt_parse.parse(row['Flight Time'])
    flight_time = time.hour*60+time.minute+time.second

    flight_class_vec = np.zeros(len(flight_class))
    flight_class_vec[flight_class.index(row['Class'])] = 1

    min_ = minimum[city.index(row['From'])][city.index(row['To'])]
    max_ = maximum[city.index(row['From'])][city.index(row['To'])]
    norm_fare = (row['Fare'])/(max_)

    feature_vec = np.concatenate((city_vec, flight_class_vec))
    feature_vec = np.concatenate((np.concatenate((feature_vec, weekday_vec)), np.asarray([duration, flight_time, ])))
#     print(X_train)
    X_train.append(feature_vec)
    Y_train.append(norm_fare)
X_train_mat = np.asarray(X_train)
Y_train_mat = np.asarray(Y_train)


X_test = []
for ind,row in test_data.iterrows():
	city_vec = np.zeros(len(city))
	city_vec[city.index(row['From'])] = 2
	city_vec[city.index(row['To'])] = 1

	age_vec = np.zeros(4)
	age = ((dt_parse.parse(row['Booking Date']) - dt_parse.parse(row['Date of Birth'])).days)/365
	if age < 12:
		age_vec[0] = 1
	elif age < 25:
		age_vec[1] = 1
	elif age < 50:
		age_vec[2] = 1
	else:
		age_vec[3] = 1

	duration = (dt_parse.parse(row['Flight Date']) - dt_parse.parse(row['Booking Date'])).days

	weekday_vec = np.zeros(7)
	weekday_ind = ((current_datetime - dt_parse.parse(row['Flight Date'])).days)%7
	weekday_vec[weekday_ind] = 1
	# print(duration*24*60)

	time = dt_parse.parse(row['Flight Time'])
	flight_time = time.hour*60+time.minute+time.second
	# print(time)
	# print(flight_time)

	flight_class_vec = np.zeros(len(flight_class))
	flight_class_vec[flight_class.index(row['Class'])] = 1

# 	feature_vec = np.concatenate((age_vec, np.concatenate((city_vec, flight_class_vec))))
	feature_vec = np.concatenate((city_vec, flight_class_vec))
	feature_vec = np.concatenate((np.concatenate((feature_vec, weekday_vec)), np.asarray([duration, flight_time])))
	# print(feature_vec)
	# print(norm_fare)
	X_test.append(feature_vec)
X_test_mat = np.asarray(X_test)


# print(len(X_train))
# print(len(Y_train))
# print(max(Y_train), min(Y_train))

# clf=LogisticRegression()
# clf = SVR()
# clf = KernelRidge(alpha=1.0)
# clf = MLPRegressor(hidden_layer_sizes=(1000, 10, )) 
clf = SVR(kernel='rbf')
split_ind = int(0.9*len(X_train))
clf.fit(X_train_mat[:split_ind],Y_train_mat[:split_ind])
Y_predict = clf.predict(X_train_mat[split_ind+1:])
y = []
for i in range(len(Y_predict)):
	if Y_train_mat[split_ind+1+i]==0:
		continue
	y.append(abs(Y_predict[i] - Y_train_mat[split_ind+1+i])/Y_train_mat[split_ind+1+i])

# y = [abs(Y_predict[i] - Y_train_mat[split_ind+1+i]/Y_train_mat[split_ind+1+i]) for i in range(len(Y_predict))] 
# y = abs(Y_predict - Y_train_mat[split_ind+1:])/Y_train_mat[split_ind+1:] 
print('LogisticRegression error_score: ', np.mean(y))


Y = []
Y_test_predict = clf.predict(X_test_mat)
# i = 0
for ind,row in test_data.iterrows():
	min_ = minimum[city.index(row['From'])][city.index(row['To'])]
	max_ = maximum[city.index(row['From'])][city.index(row['To'])]
	Y.append(Y_test_predict[ind]*(max_))

with open('/home/krullun/Documents/coding/JPMorgan/Problem_1/out.txt', 'w') as f:
    for i in Y:
        f.write("%f\n" % i)

f.close()
print(Y)

('LogisticRegression error_score: ', 0.6847834055618631)
[3777.3554127487578, 5412.956059825074, 7414.213017709655, 8180.65234860276, 18441.340215281452, 19749.20235344851, 6080.671128722624, 4504.212603595421, 7620.971993167502, 4701.518903356169, 16181.570664220988, 6016.295458077446, 5999.331740237001, 10969.76698322729, 3627.734564004253, 5716.179578791536, 7575.955971373148, 15216.69349240006, 8694.08092209223, 9880.409621399654, 4639.624024145312, 8917.67816447165, 17509.847661248972, 5007.0512279976465, 13506.34062710644, 2938.1212004057975, 9051.124992283503, 7593.617546184738, 9599.752114353469, 19872.36175245531, 6469.0664588788095, 5791.048549648598, 7438.385543760391, 7674.111041913595, 19973.75493368858, 8045.261829154432, 13192.849155863594, 6892.06675867765, 6639.283461420119, 15800.110561110543, 9579.336882535035, 3816.2395421347073, 6232.593105824712, 8054.780978795254, 4873.971268323707, 11110.768249146562, 9128.758376461348, 10538.139416373768, 9015.978594747932, 697